In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import json
import pickle
import pandas as pd
import numpy as np
from pyontutils.utils import * 
from pyontutils.core import *
from pyontutils.closed_namespaces import *
import rdflib
from collections import defaultdict

In [4]:
iri_to_family_iris = json.load(open('/home/troy/Desktop/interlexutils/nif-ilx-comparator/dump/iri-to-family.json','r'))
iri_to_ilx = json.load(open('/home/troy/Desktop/interlexutils/nif-ilx-comparator/dump/iri-to-ilx.json','r'))

In [5]:
g_ilx = makeGraph('', graph=rdflib.Graph().parse('../Interlex.ttl', format='turtle'))
#g_ilx = makeGraph('', graph=pickle.load(open('../Interlex.pickle', 'rb')))
g_qnames_purpose_only = makeGraph('', graph=rdflib.Graph().parse('../Interlex_qnames.ttl', format='turtle'))
g_nif = makeGraph('', graph=pickle.load(open('../NIF-ALL.pickle', 'rb')))

In [1]:
#check its value aka NIFRID then if it fails, check if the fails are in owl.subClassOf and then rdf.first. If there are still some that fail then look into why it fails 
#and see if its just a predicate you need to input intead of NIFRID or a different scope
'''
graph = makeGraph('', graph=rdflib.Graph().parse('../../work/NIF-Ontology/ttl/bridge/uberon-bridge.ttl', format='turtle'))

def check_restrictions(graph):
    r=Restriction(NIFRID.has_proper_part)
    pred_passed = set(r.parse(graph=graph.g))

    r=Restriction(owl.subClassOf)
    sub_passed = set(r.parse(graph=graph.g))

    r=Restriction(rdf.first)
    list_passed = set(r.parse(graph=graph.g))

    return pred_passed & sub_passed & list_passed

check_restrictions(graph)
'''

"\ngraph = makeGraph('', graph=rdflib.Graph().parse('../../work/NIF-Ontology/ttl/bridge/uberon-bridge.ttl', format='turtle'))\n\ndef check_restrictions(graph):\n    r=Restriction(NIFRID.has_proper_part)\n    pred_passed = set(r.parse(graph=graph.g))\n\n    r=Restriction(owl.subClassOf)\n    sub_passed = set(r.parse(graph=graph.g))\n\n    r=Restriction(rdf.first)\n    list_passed = set(r.parse(graph=graph.g))\n\n    return pred_passed & sub_passed & list_passed\n\ncheck_restrictions(graph)\n"

In [6]:
""" Delete all Defaults """
ns_to_del = []
for namespace in g_nif.namespaces:
    if 'default' in namespace.split(':')[0]:
        ns_to_del.append(namespace)
for ns in ns_to_del:
    g_nif.del_namespace(ns)
    
""" Create new namespaces; old are messed up """
for s, ns in g_nif.namespaces.items():
    g_nif.del_namespace(s)
    g_nif.add_namespace(s, rdflib.Namespace(str(ns)))

""" Reset blasted cache """
for _ in range(10):
    g_nif.g.namespace_manager.reset()
    
""" Fill ilx qname to fix the existing ids """
for s, ns in g_nif.namespaces.items():
    if not g_qnames_purpose_only.namespaces.get(s):
        g_qnames_purpose_only.add_namespace(s, rdflib.Namespace(str(ns)))

In [7]:
edge_cases = {
    #Definitions
    'definition:' : 'definition:',
    'skos:definition' : 'definition:',
    'definition' : 'definition:',
    'NIFRID:birnlexDefinition' : 'definition:',
    'NIFRID:externallySourcedDefinition' : 'definition:',
    #ExistingIds
    'ilxtr:existingIds' : 'ilxtr:existingIds',  
    #LABELS
    'rdfs:label' : 'rdfs:label',
    'skos:prefLabel' : 'rdfs:label',
    #SUPERCLASSES
    'rdfs:subClassOf' : 'rdfs:subClassOf',
    #SYNONYMS
    'oboInOwl:hasExactSynonym' : 'NIFRID:synonym',
    #'oboInOwl:hasNarrowSynonym' : 'NIFRID:synonym',
    #'oboInOwl:hasBroadSynonym' : 'NIFRID:synonym',
    'oboInOwl:hasRelatedSynonym' : 'NIFRID:synonym',
    'go:systematic_synonym' : 'NIFRID:synonym',
    'NIFRID:synonym' : 'NIFRID:synonym',
    #TYPE
    'rdf:type':'rdf:type',   
}

full = {
    #'':None,  # safety (now managed directly in the curies file)
    #'EHDAA2':'http://purl.obolibrary.org/obo/EHDAA2_',  # FIXME needs to go in curie map?

    'hasRole':'http://purl.obolibrary.org/obo/RO_0000087',
    'inheresIn':'http://purl.obolibrary.org/obo/RO_0000052',
    'bearerOf':'http://purl.obolibrary.org/obo/RO_0000053',
    'participatesIn':'http://purl.obolibrary.org/obo/RO_0000056',
    'hasParticipant':'http://purl.obolibrary.org/obo/RO_0000057',
    'adjacentTo':'http://purl.obolibrary.org/obo/RO_0002220',
    'derivesFrom':'http://purl.obolibrary.org/obo/RO_0001000',
    'derivesInto':'http://purl.obolibrary.org/obo/RO_0001001',
    'agentIn':'http://purl.obolibrary.org/obo/RO_0002217',
    'hasAgent':'http://purl.obolibrary.org/obo/RO_0002218',
    'containedIn':'http://purl.obolibrary.org/obo/RO_0001018',
    'contains':'http://purl.obolibrary.org/obo/RO_0001019',
    'locatedIn':'http://purl.obolibrary.org/obo/RO_0001025',
    'locationOf':'http://purl.obolibrary.org/obo/RO_0001015',
    'toward':'http://purl.obolibrary.org/obo/RO_0002503',

    'replacedBy':'http://purl.obolibrary.org/obo/IAO_0100001',
    'hasCurStatus':'http://purl.obolibrary.org/obo/IAO_0000114',
    'definition':'http://purl.obolibrary.org/obo/IAO_0000115',
    'editorNote':'http://purl.obolibrary.org/obo/IAO_0000116',
    'termEditor':'http://purl.obolibrary.org/obo/IAO_0000117',
    'altTerm':'http://purl.obolibrary.org/obo/IAO_0000118',
    'defSource':'http://purl.obolibrary.org/obo/IAO_0000119',
    'termsMerged':'http://purl.obolibrary.org/obo/IAO_0000227',
    'obsReason':'http://purl.obolibrary.org/obo/IAO_0000231',
    'curatorNote':'http://purl.obolibrary.org/obo/IAO_0000232',
    'importedFrom':'http://purl.obolibrary.org/obo/IAO_0000412',

    'partOf':'http://purl.obolibrary.org/obo/BFO_0000050',
    'hasPart':'http://purl.obolibrary.org/obo/BFO_0000051',
}

normal = {
    'ILX':'http://uri.interlex.org/base/ilx_',
    'ilx':'http://uri.interlex.org/base/',
    'ilxr':'http://uri.interlex.org/base/readable/',
    'ilxtr':'http://uri.interlex.org/tgbugs/uris/readable/',
    # for obo files with 'fake' namespaces, http://uri.interlex.org/fakeobo/uris/ eqiv to purl.obolibrary.org/
    'fobo':'http://uri.interlex.org/fakeobo/uris/obo/',

    'PROTEGE':'http://protege.stanford.edu/plugins/owl/protege#',
    'ILXREPLACE':'http://ILXREPLACE.org/',
    'TEMP': interlex_namespace('temp/uris'),
    'FIXME':'http://FIXME.org/',
    'NIFTTL':'http://ontology.neuinfo.org/NIF/ttl/',
    'NIFRET':'http://ontology.neuinfo.org/NIF/Retired/NIF-Retired.owl#',
    'NLXWIKI':'http://neurolex.org/wiki/',
    'dc':'http://purl.org/dc/elements/1.1/',
    'dcterms':'http://purl.org/dc/terms/',
    'dctypes':'http://purl.org/dc/dcmitype/',  # FIXME there is no agreement on qnames
    # FIXME a thought: was # intentionally used to increase user privacy? or is this just happenstance?
    'nsu':'http://www.FIXME.org/nsupper#',
    'oboInOwl':'http://www.geneontology.org/formats/oboInOwl#',
    'owl':'http://www.w3.org/2002/07/owl#',
    'ro':'http://www.obofoundry.org/ro/ro.owl#',
    'skos':'http://www.w3.org/2004/02/skos/core#',
    'rdf':'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
    'rdfs':'http://www.w3.org/2000/01/rdf-schema#',
    'prov':'http://www.w3.org/ns/prov#',
}

expo = {
    'ilxtr:existingId':'ilxtr:identifier',
    'oboInOwl:hasAlternativeId':'ilxtr:identifier',
    'NIFRID:isReplacedByClass':'replacedBy:',
    'skos:editorialNote' : 'editorNote:',
    'ncbitaxon:has_rank' : 'NIFRID:hasTaxonRank',
}
#Maybe a 5th category which is "labels/synonyms with parens in them" 
#would be a way to see whether that is a good filter would help too.

#extras = {**full, **normal}
extras = full

qilx = g_ilx.qname
qnif = g_nif.qname
#qnif = g_ilx.qname
qall = g_qnames_purpose_only.qname

edge_cases = {**{qnif(v):k+':' for k,v in extras.items()}, **edge_cases, **expo}

In [8]:
interlex_with_mods = defaultdict(set)
nif_with_mods = defaultdict(set)

#changed to list to have a ranking system within the data to make it more complicated with bloat data.
rank_list = []
data = defaultdict(list)
def rank(pad, rank_list):
    if not rank_list: return pad
    score = []
    rank_list = rank_list[::-1]
    for item in pad:
        if item in rank_list:
            score.append((rank_list.index(item), item))
        else:
            score.append((-1, item))
    return [v[1] for v in sorted(score)[::-1]]

for subj in g_ilx.g.subjects(rdflib.RDF.type, rdflib.OWL.Class):
    
    #if qilx(subj) != 'ILX:0102930': continue
    
    shared_iris = set()
    ilx_mod_conversion = {}
    nif_mod_conversion = {}
    
    for pred, obj in g_ilx.g.predicate_objects(subject=subj):
        
        '''Needs traversal convert superclass ilx to its respected exisiting ids'''
        if qilx(pred) == 'rdfs:subClassOf': 
            superclasses_iris = iri_to_family_iris[str(obj)]
            for superclasses_iri in superclasses_iris:
                tup = (qilx(pred), ' '.join(superclasses_iri.lower().strip().split()))
                ilx_mod_conversion[tup] = (qilx(pred), superclasses_iri)
                interlex_with_mods[qilx(subj)].add(tup)    
        else:        
            tup = (qnif(pred), ' '.join(str(obj).lower().strip().split()))
            ilx_mod_conversion[tup] = (qilx(pred), str(obj))
            interlex_with_mods[qilx(subj)].add(tup)
        
        '''Build nif when ilx exisiting id found in nif'''
        if qilx(pred) == 'ilxtr:existingIds':
            for p, o in g_nif.g.predicate_objects(subject=obj): 
                shared_iris.add(obj)
                tup = (qnif(p), ' '.join(str(o).lower().strip().split()))
                if edge_cases.get(qnif(p)):
                    nif_with_mods[qnif(obj)].add((edge_cases[tup[0]], tup[1]))
                    nif_mod_conversion[(edge_cases[tup[0]], tup[1])] = (qnif(p), str(o))
                else:
                    nif_with_mods[qnif(obj)].add(tup)
                    nif_mod_conversion[tup] = (qnif(p), str(o))
                    
    '''Comparator btw current ilx term and each nif term sharing ilx existing id'''      
    for shared_iri in rank(shared_iris, rank_list):
        #print(shared_iris)
        nif = nif_with_mods[qnif(shared_iri)]
        ilx = interlex_with_mods[qilx(subj)]
        
        i = ilx - nif
        n = nif - ilx
        b = nif & ilx
        
        def formatter(current_set, mod_conversion):
            return sorted([(p,(o)) for p,o in [mod_conversion[po] for po in current_set]])
        
        data[qall(subj)].append({
            qall(shared_iri): {
                'ilx_only'     : formatter(i, ilx_mod_conversion),
                'nif_only'     : formatter(n, nif_mod_conversion),
                'both_contain' : formatter(b, nif_mod_conversion),
            }
        })    

outf = '/home/troy/Desktop/interlexutils/ontology-interlex/dump/moderate-nif-ilx-comparator-extras.json'
with open(outf, 'w') as outfile:
    json.dump(data, outfile, indent=4)
    
#ex for short hand vi -e 
#-s so it doesnt print file details
#-cwq is command-save-quit
#use $ to specify variable
!ex -s +'g/\[[\ \n]\+"/j4' -cwq $outf
print("COMPLETE")

FileNotFoundError: [Errno 2] No such file or directory: '/home/troy/Desktop/interlexutils/ontology-interlex/dump/moderate-nif-ilx-comparator-extras.json'

In [28]:
for pred, obj in g_nif.g.predicate_objects(subject=rdflib.term.URIRef('http://uri.neuinfo.org/nif/nifstd/nlx_157906')):
    print(pred, obj)

http://www.w3.org/2000/01/rdf-schema#label identifier resolution
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Class
http://www.w3.org/2000/01/rdf-schema#subClassOf http://uri.neuinfo.org/nif/nifstd/nlx_res_20090416


## Discover

In [ ]:
comparator_data = json.load(open('dump/moderate-nif-ilx-comparator-extras.json', 'r'))

In [ ]:
import re

def cv(string):
    return ' '.join(string.lower().strip().split())

hits = set()
new_comparator_data = comparator_data.copy()
count_per_id = 0
count_per_element = 0

for ilx_identifer, shared_iris in comparator_data.items():
    for i, shared_iri in enumerate(shared_iris):
        for iri, inb in shared_iri.items():
            expo = {'same':[], 'different':[]}
            for ilx_pred, ilx_obj in inb['ilx_only']:
                for nif_pred, nif_obj in inb['nif_only']:
                    
                    ''' diff logic here '''
                    for test in [r'definition', r'synonym', r'label']: #for each test case given; keeps noise out
                        if re.search(test, cv(ilx_pred)) and re.search(test, cv(nif_pred)): #and nif_pred != 'skos:definition':
                            if cv(ilx_obj) == cv(nif_obj):
                                count_per_element += 1
                                expo['same'].append([(test, nif_pred), nif_obj])
                                print(test, ilx_pred, nif_pred)
                            else:
                                expo['different'].append([(test, nif_pred), [ilx_obj, nif_obj]])
            
            if expo: count_per_id += 1
            #print(expo)
            new_comparator_data[ilx_identifer][i][iri]['experimental'] = expo #[(),(),()]

outf = 'dump/expo-moderate-nif-ilx-comparator-extras2.json'
with open(outf, 'w') as outfile:
    json.dump(new_comparator_data, outfile, indent=4)
!ex -s +'g/\[[\ \n]\+"/j4' -cwq $outf
print(count_per_id)
print(count_per_element)

In [17]:
a={'key':'value'}
key, value = a.keys(), a.values()
print(key, value)

dict_keys(['key']) dict_values(['value'])


## random possible equals

ILX:0106722
rdfs:subClassOf == oboInOwl:id 

ILX:0104473
definition: == dc:description 

ILX:0101393
NIFRID:synonym == NIFRID:taxonomicCommonName 

ILX:0101464
definition:
NIFRID:externallySourcedDefinition

ILX:0102938
definition: A solubility which is relatively low.
obo:IAO_0000115 A solubility which is relatively low. 

ILX:0104395
rdfs:subClassOf PATO:0001444
obo:IAO_0100001 PATO:0001444 

ILX:0109258
definition: == obo:UBPROP_0000001 

ILX:0108780
rdfs:label PH-sensitive electrode recording protocol
skos:prefLabel pH-sensitive electrode recording protocol

ILX:0109341
rdfs:label Primate
skos:altLabel primate 

ILX:0101385
rdfs:subClassOf NCBITaxon:7089
oboInOwl:id NCBITaxon:7089 

ILX:0101527
rdfs:subClassOf == NIFRID:hasFormerParentClass 

ILX:0111998
rdfs:subClassOf == owl:equivalentClass

In [4]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet as wn
 
def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None

def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """
    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))

    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0
 
    # For each word in the first sentence
    for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
            
        best_score=[synset.path_similarity(ss) for ss in synsets2 if synset.path_similarity(ss)]
    
        # Check that the similarity could have been computed
        if best_score:
            score += max(best_score)
            count += 1
    
    # Average the values
    if count > 0:
        score /= count
    else:
        score = 0

    return score